In [8]:
#Libraries are imported
import pandas as pd
import numpy as np
import math
import time
import random
import matplotlib.pyplot as plt
import statistics

In [9]:
#City coordinates are imported as x and y
coord=pd.read_csv("tsp.csv",names=["x","y"],header= None, ) 

In [10]:
#Calculate_distance function that calculates distances between 2 given city with euclidean distance is intoduced
def calculate_distance(coord,city1,city2):
    distance=math.sqrt(np.sum((coord.iloc[city1]-coord.iloc[city2])**2))
    return distance

In [11]:
#create_dist_matrix function creating a matrix that stores distances between all cities to decrease cpu time
def create_dist_matrix(coord):
    distance = np.zeros((len(coord),len(coord)))
    for city1 in range(51):
        for city2 in range(51):
            if city1 != city2:
                distance[city1 ,city2] = calculate_distance(coord, city1, city2)
    return distance

In [12]:
#find_distance is a function that calculate distance for a given solution array (v_current)
def find_distance(v_current,distance):
    mesafe=0
    for i in range(49):
        mesafe=mesafe+distance[v_current[i],v_current[i+1]]
    mesafe=mesafe+distance[v_current[0],v_current[50]]
    return mesafe    

In [13]:
#generate_randomsolution is a function that creating initial random solution
def generate_randomsolution():
    res = random.sample(range(0, 51),51)
    return res

In [14]:
#city_swap is a function that changes the place of two random point in queue at the current solution array
def city_swap(v_current):
    i,j = random.sample(range(0, 50), 2)
    v_new=v_current.copy()
    v_new[i], v_new[j] = v_current[j], v_current[i]
    return v_new

In [15]:
def simulated_annealing(coord,T,halting_criterion,alpha):
    h=0 #iteraion number
    max_iter = 300
    v_current=generate_randomsolution() #creating initial solution array
    eval_c=find_distance(v_current,distance) #finding initial solution distance
    while halting_criterion<T: #halting criterion
        inner_h = 0
        while True:
            inner_h=inner_h+1
            v_new=city_swap(v_current)
            eval_n=find_distance(v_new,distance)
            if eval_c>eval_n: #accepts better solution as current
                v_current=v_new
                eval_c=find_distance(v_current,distance)
                
            else: #accepts worse solution with an probability
                if random.random()<math.exp((eval_c-eval_n)/T):
                   v_current=v_new
                   eval_c=find_distance(v_current,distance)
                   
            if (eval_c-eval_n)<=1 or inner_h >= max_iter: #termination condition
                break       
        h=h+1
        T=T*alpha
        
    return v_current,eval_c,h     


In [10]:
#exploring parameters
distance=create_dist_matrix(coord)
alphas = np.linspace(0.9905,0.9999,num=10)
Ts = np.linspace(100,2000,num=10)
Haltings = np.linspace(0.05,2.05,num=11)
Trials=[]
for i in alphas:
    for t in Ts:
        for cr in Haltings:
            ort_s=[]
            ort_r=[]
            for r in range (5):
                start=time.time()
                v_current,eval_c,h=simulated_annealing(coord,t,cr,i)
                end=time.time()
                ort_r.append(end-start)
                ort_s.append(eval_c)
            ort_s_use=statistics.mean(ort_s)
            st_dev=statistics.stdev(ort_s)
            ort_r_use=statistics.mean(ort_r)
            Trials.append([i,ort_s_use,st_dev,t,cr,h,ort_r_use])
df = pd.DataFrame(Trials,columns=['Alpha',
                                  'Mean Solution Distance',"Solution St Deviation",'T',
                                  'Halting Criterion',
                                  'Iteration Number','Mean Computational Time'])
df.sort_values('Mean Solution Distance',ascending=True)

,Alpha,Mean Solution Distance,Solution St Deviation,T,Halting Criterion,Iteration Number,Mean Computational Time
1067,0.999900,467.432481,10.022624,1577.777778,0.05,103590,3.082961
1023,0.999900,469.353310,13.938752,733.333333,0.05,95929,2.788084
1005,0.999900,469.907183,15.359798,311.111111,0.85,59024,1.810898
1069,0.999900,471.870020,19.192829,1577.777778,0.45,81619,2.598415
1046,0.999900,471.996512,10.991404,1155.555556,0.25,84383,2.585694
...,...,...,...,...,...,...,...
85,0.990500,984.595292,39.979082,1577.777778,1.65,719,0.024823
31,0.990500,987.731321,51.286309,522.222222,1.85,592,0.020219
21,0.990500,988.128528,42.708707,311.111111,2.05,527,0.017416
119,0.991544,992.195132,26.050716,100.000000,1.85,470,0.014413


In [11]:
dataframe_last = df.sort_values('Mean Solution Distance',ascending=True)
dataframe_last.to_excel("output_satsp3.xlsx")

In [20]:
distance=create_dist_matrix(coord)
start=time.time()
best=simulated_annealing(coord,1577,0.05,0.9999)
end=time.time()
print("time:",end-start,"route:",best[0],"distance:",best[1],"iteration:",best[2])

time: 3.025745391845703 route: [30, 25, 7, 47, 22, 6, 42, 23, 24, 12, 40, 39, 18, 41, 3, 17, 13, 5, 26, 50, 45, 11, 46, 16, 36, 43, 14, 44, 32, 38, 9, 48, 4, 37, 8, 29, 33, 49, 20, 28, 1, 15, 10, 31, 0, 21, 2, 19, 34, 35, 27] distance: 429.47499372567853 iteration: 103585
